# 패키지

In [1]:
import MSRM

# 모델 생성

In [2]:
# 이미지 인식 모델 생성
model = MSRM.SymbolDetector(
    ['sheet_1.png', 'sheet_2.png', 'sheet_3.png', 'sheet_4.png'],
    note_confidences=0.5,   # 확률이 note_confidences가 넘는 이미지들만 추출
    note_iou_threshold=0.1  # 동일한 위치의 악상기호가 겹치는 비율이 note_iou_threshold이상이면 제거 작업
)

# 클래스 레이블 준비

In [3]:
# 클래스별 이름들
class_names=[               
    'staff',
    'clef',
    'key',
    'measure',
    'rest',
    'time',
    'note', 
    'accidental',
    'articulation',
    'dynamic',
    'glissando',
    'octave',
    'ornament',
    'repetition',
    'tie'
]

# 모델 인식 (과정 보여주기)

In [ ]:
# 악상기호 인식 시작
images, datas = model.detact(preview=True)

# 모델 인식 (결과 보여주기)

In [ ]:
# 악상기호 인식 시작
images, datas = model.detact(preview=False)

# 미리보기
for image, data in zip(images, datas):
    MSRM.preview(
        image,                      # 이미지
        data[0], data[1], data[2],  # 바운딩 박스 데이터: boxes, confidence, class
        class_names=class_names,    # 클래스별 이름들
        window_height=900   # 미리보기할 이미지 창의 높이
    )

# 모델 인식 (결과 저장)

In [ ]:
# 악상기호 인식 시작
images, datas = model.detact(preview=False)

# 미리보기
count = 0
for image, data in zip(images, datas):
    MSRM.save(
        image,                      # 이미지
        str(count)+'.png',          # 저장할 이름
        data[0], data[1], data[2],  # 바운딩 박스 데이터: boxes, confidence, class
        class_names=class_names,    # 클래스별 이름들
    )
    count += 1 